# Requirements

In [29]:
import tenseal as ts # pip install tenseal
from deepface import DeepFace #!pip install deepface
import base64
from deepface.commons import distance as dst


# Finding embeddings (Client Side)
We are going to find vector representations of facial images. This will be done in the client side.

In [3]:
img1_path = "photo.jpeg"
img2_path = "new photo.jpeg"

In [4]:
img1_embedding=DeepFace.represent(img1_path, model_name='Facenet')
img2_embedding=DeepFace.represent(img2_path, model_name='Facenet')

In [9]:
len(img1_embedding)

1

# Commons

In [33]:
def write_data(file_name, file_content):
    
    if type(file_content) == bytes:
        #bytes to base64
        
        file_content = base64.b64encode(file_content)
    
    with open(file_name, 'wb') as f: 
        f.write(file_content)

def read_data(file_name):
    with open(file_name, "rb") as f:
        file_content = f.read()
    
    #base64 to bytes
    return base64.b64decode(file_content)

# Initialize (Client Side)

In [34]:
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree = 8192, coeff_mod_bit_sizes = [60, 40, 40, 60])

In [35]:
context.generate_galois_keys()
context.global_scale = 2**40

In [36]:
secret_context = context.serialize(save_secret_key = True)
write_data("secret.txt", file_content=secret_context)

In [37]:
type(secret_context)

bytes

In [38]:
context.make_context_public()
public_context=context.serialize()
write_data(file_name="public.txt",file_content=public_context)

In [39]:
del context, secret_context, public_context

# Encryption
We are going to apply homomorphic encryption to facial embeddings. This will be done in the client side.

Then, homomorphic encrypted facial embeddings will be stored in the cloud.

In [48]:
context = ts.context_from(read_data("secret.txt"))
enc_v1 = ts.ckks_vector(context, img1_embedding)
enc_v2 = ts.ckks_vector(context, img2_embedding)

TypeError: input isn't a well structured tensor

In [49]:
enc_v1_proto = enc_v1.serialize()
enc_v2_proto = enc_v2.serialize()

NameError: name 'enc_v1' is not defined

In [50]:
write_data("enc_v1.txt", enc_v1_proto)
write_data("enc_v2.txt", enc_v2_proto)

NameError: name 'enc_v1_proto' is not defined

In [51]:
del context, enc_v1, enc_v2, enc_v1_proto, enc_v2_proto

NameError: name 'enc_v1' is not defined

# Calculations
Once homomorphic encrypted facial embeddings stored in the cloud, we are able to make calculations on encrypted data.

Notice that we just have public key here and we don't have secret key.

In [52]:
context = ts.context_from(read_data("public.txt"))

In [53]:
enc_v1_proto = read_data("enc_v1.txt")
enc_v2_proto = read_data("enc_v2.txt")

In [54]:
enc_v1 = ts.lazy_ckks_vector_from(enc_v1_proto)
enc_v1.link_context(context)

enc_v2 = ts.lazy_ckks_vector_from(enc_v2_proto)
enc_v2.link_context(context)

In [55]:
euclidean_squared = enc_v1 - enc_v2
euclidean_squared = euclidean_squared.dot(euclidean_squared)

In [56]:
write_data("euclidean_squared.txt", euclidean_squared.serialize())

In [57]:
#we must not decrypt the homomorphic encrypted euclidean squared value in this stage
#because we don't have the secret key. check this operation. it should throw an exception!

try:
    euclidean_squared.decrypt()
except Exception as err:
    print("Exception: ", str(err))

Exception:  the current context of the tensor doesn't hold a secret_key, please provide one as argument


In [58]:
del context, enc_v1_proto, enc_v2_proto, enc_v1, enc_v2, euclidean_squared

# Decryption
Once homomorphic encrypted euclidean squared value found in the cloud, we are going to retrieve it to the client side.

Client can decrypt it because we have the secret key in the client side.



In [59]:
context = ts.context_from(read_data("secret.txt"))

In [60]:
euclidean_squared_proto = read_data("euclidean_squared.txt")

In [61]:
euclidean_squared = ts.lazy_ckks_vector_from(euclidean_squared_proto)
euclidean_squared.link_context(context)

In [62]:
euclidean_squared_plain = euclidean_squared.decrypt()[0]

In [63]:
euclidean_squared_plain

-1.123258424983683e+19

In [64]:
if euclidean_squared_plain < 100:
    print("they are same person")
else:
    print("they are different persons")

they are same person


# Validation
What if euclidean distance calculation is done in the client side always? Result should be same!

In [67]:
distance = dst.findEuclideanDistance(img1_embedding, img2_embedding)


TypeError: unsupported operand type(s) for -: 'dict' and 'dict'

In [66]:
print("euclidean squared - tradational: ", distance*distance)
print("euclidean squared - homomorphic: ", euclidean_squared_plain)

NameError: name 'distance' is not defined

In [68]:
#check the difference is acceptable
abs(distance * distance - euclidean_squared_plain) < 0.00001

NameError: name 'distance' is not defined